# Sample Fitness Polar Dataset
* Análise dos dados de atividades esportivas
* Módulo I - Importação dos dados dos arquivos JSON e geração de arquivos CSV

In [1]:
# Importação dos pacotes
import os, json
import pandas as pd
import numpy as np
import glob

In [2]:
# Inicializa indices
id_atividade = 0
id_exercicio = 0
id_informacao = 0
id_zona_heart = 0
id_zona_speed = 0
id_amostra_heart = 0
id_amostra_speed = 0
id_amostra_distance = 0
id_amostra_altitude = 0
id_amostra_route = 0


In [3]:
#Inicializa dataframes
atividades        = pd.DataFrame(columns=['id_atividade','exportVersion','latitude','longitude','startTime','stopTime','distance','name','timeZoneOffset','kiloCalories'])
exercicios        = pd.DataFrame(columns=['id_exercicio','id_atividade' ,'startTime','stopTime','distance','sport','latitude','longitude','timeZoneOffset','kiloCalories','cadence_avg','cadence_max'])

informacoes       = pd.DataFrame(columns=['id_informacao','id_atividade' ,'dateTime','sex','birthday','height, cm','weight, kg','vo2Max','maximumHeartRate','restingHeartRate','aerobicThreshold', 'anaerobicThreshold','functionalThresholdPower'])

zonas_heart       = pd.DataFrame(columns=['id_exercicio','id_atividade' ,'id_zona_heart','lowerLimit','higherLimit','zoneIndex'])
zonas_speed       = pd.DataFrame(columns=['id_exercicio','id_atividade' ,'id_zona_speed','lowerLimit','higherLimit','zoneIndex'])
amostras_heart    = pd.DataFrame(columns=['id_exercicio','id_atividade' ,'id_amostra_heart','dateTime','value'])
amostras_speed    = pd.DataFrame(columns=['id_exercicio','id_atividade' ,'id_amostra_speed','dateTime','value'])
amostras_distance = pd.DataFrame(columns=['id_exercicio','id_atividade' ,'id_amostra_distance','dateTime','value'])
amostras_altitude = pd.DataFrame(columns=['id_exercicio','id_atividade' ,'id_amostra_altitude','dateTime','value'])
amostras_route    = pd.DataFrame(columns=['id_exercicio','id_atividade' ,'id_amostra_route','dateTime','latitude','longitude'])


In [4]:
# definição da função de conversão para float (testa se é string e se é possível a conversão)
def flt(txt):  
    if type(txt) == str:
        try :  
            return float(txt) 
        except :
            return 0
    else:
        return txt
    

In [5]:
# Arredonda valores retornados pela funcao flt
def arred(nr):
    try:
        return round(flt(nr),1)
    except:
        return flt(nr)

In [6]:
#Preparação para leitura dos arquivos

pd.set_option('display.max_columns', None)
path_to_json = '/home/walter/Downloads/polar-sample-database-master/dataset/' 
json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)

In [7]:
# Inserção dos dados nos dataframes

for file in file_list:
    with open(file) as f:
      arquivo = json.load(f)
      id_atividade = id_atividade + 1
      print(id_atividade)
      # Leitura de informações sobre a atividade
      atividades = atividades.append({'id_atividade':id_atividade,'exportVersion':arquivo.get('exportVersion'),'latitude':arred(arquivo.get('latitude')),'longitude':arred(arquivo.get('longitude')),'startTime':pd.to_datetime(arquivo.get('startTime')),'stopTime':pd.to_datetime(arquivo.get('stopTime')),'distance':flt(arquivo.get('distance'))}, ignore_index=True)

      informacao = arquivo.get('physicalInformationSnapshot')
      if informacao:
        id_informacao = id_informacao + 1
        # Captura de dados sobre o atleta
        informacoes = informacoes.append({ 'id_informacao':id_informacao,'id_atividade':id_atividade,'dateTime':pd.to_datetime(informacao.get('dateTime')),'sex':informacao.get('sex'),'birthday':informacao.get('birthday'),'height, cm':flt(informacao.get('height, cm')),'weight, kg':flt(informacao.get('weight, kg')),'vo2Max':flt(informacao.get('vo2Max')),'maximumHeartRate':flt(informacao.get('maximumHeartRate')),'restingHeartRate':flt(informacao.get('restingHeartRate')),'aerobicThreshold':flt(informacao.get('aerobicThreshold')), 'anaerobicThreshold':flt(informacao.get('anaerobicThreshold')),'functionalThresholdPower':flt(informacao.get('functionalThresholdPower'))}, ignore_index=True)  

      for exercicio in arquivo['exercises']:
        id_exercicio = id_exercicio + 1
        cadencia = exercicio.get('cadence')
        avg = ''
        max = ''
        if cadencia:
            avg = cadencia.get('avg')
            max = cadencia.get('max')
        # Captura de dados sobre o exercício
        exercicios = exercicios.append({ 'id_exercicio':id_exercicio,'id_atividade':id_atividade,'startTime':pd.to_datetime(exercicio.get('startTime')),'stopTime':pd.to_datetime(exercicio.get('stopTime')),'distance':flt(exercicio.get('distance')),'sport':exercicio.get('sport',''),'latitude':arred(exercicio.get('latitude')),'longitude':arred(exercicio.get('longitude')),'cadence_avg':flt(avg),'cadence_max':flt(max)}, ignore_index=True)  

        zona_heart_rate = exercicio['zones'].get('heart_rate')
        if zona_heart_rate:
            for heart in zona_heart_rate:
                id_zona_heart = id_zona_heart + 1
                # Captura de dados sobre as zonas de treinamento de acordo com os batimentos cardiacos
                zonas_heart = zonas_heart.append({'id_exercicio':id_exercicio,'id_atividade':id_atividade,'id_zona_heart':id_zona_heart,'lowerLimit':flt(heart.get('lowerLimit')),'higherLimit':flt(heart.get('higherLimit')),'zoneIndex':heart.get('zoneIndex')}, ignore_index=True)
                
        zona_speed_rate = exercicio['zones'].get('speed')
        if zona_speed_rate:
            for speed in zona_speed_rate:
                id_zona_speed = id_zona_speed + 1
                # Captura de dados sobre as zonas de treinamento de acordo com a velocidade
                zonas_speed = zonas_speed.append({'id_exercicio':id_exercicio,'id_atividade':id_atividade,'id_zona_speed':id_zona_speed,'lowerLimit':flt(speed.get('lowerLimit')),'higherLimit':flt(speed.get('higherLimit')),'zoneIndex':speed.get('zoneIndex')}, ignore_index=True)
                
        amostra_speed_rate = exercicio['samples'].get('speed')  #['speed', 'distance', 'altitude', 'recordedRoute','heartRate']
        if amostra_speed_rate:
            for speed in amostra_speed_rate:
                id_amostra_speed = id_amostra_speed + 1
                # Captura de dados sobre a velocidade durante as atividades
                amostras_speed = amostras_speed.append({'id_exercicio':id_exercicio,'id_atividade':id_atividade,'id_amostra_speed':id_amostra_speed,'dateTime':pd.to_datetime(speed.get('dateTime')),'value':flt(speed.get('value'))}, ignore_index=True)            
            
            
        amostra_altitude = exercicio['samples'].get('altitude')  #['speed', 'distance', 'altitude', 'recordedRoute','heartRate']
        if amostra_altitude:
            for altitude in amostra_altitude:
                id_amostra_altitude = id_amostra_altitude + 1
                # Captura de dados sobre a altitude durante as atividades
                amostras_altitude = amostras_altitude.append({'id_exercicio':id_exercicio,'id_atividade':id_atividade,'id_amostra_altitude':id_amostra_altitude,'dateTime':pd.to_datetime(altitude.get('dateTime')),'value':flt(altitude.get('value'))}, ignore_index=True)            
                        
            
        amostra_heart_rate = exercicio['samples'].get('heartRate')  #['speed', 'distance', 'recordedRoute','heartRate']
        if amostra_heart_rate:
            for heart in amostra_heart_rate:
                    id_amostra_heart = id_amostra_heart + 1
                    # Captura de dados sobre os batimentos cardíacos durante as atividades
                    amostras_heart = amostras_heart.append({'id_exercicio':id_exercicio,'id_atividade':id_atividade,'id_amostra_heart':id_amostra_heart,'dateTime':pd.to_datetime(heart.get('dateTime')),'value':flt(heart.get('value'))}, ignore_index=True)                

        amostra_distance = exercicio['samples'].get('distance')  #['speed', 'distance', 'recordedRoute','heartRate']
        if amostra_distance:
            for distance in amostra_distance:
                id_amostra_distance = id_amostra_distance + 1
                # Captura de dados sobre a distancia percorrida durante as atividades
                amostras_distance = amostras_distance.append({'id_exercicio':id_exercicio,'id_atividade':id_atividade,'id_amostra_distance':id_amostra_distance,'dateTime':pd.to_datetime(distance.get('dateTime')),'value':flt(distance.get('value'))}, ignore_index=True)                

        amostra_route = exercicio['samples'].get('recordedRoute')  #['speed', 'distance', 'recordedRoute','heartRate']
        if amostra_route:
            for route in amostra_route:
                id_amostra_route = id_amostra_route + 1
                # Captura de dados sobre a rota percorrida durante as atividades
                amostras_route = amostras_route.append({'id_exercicio':id_exercicio,'id_atividade':id_atividade,'id_amostra_route':id_amostra_route,'dateTime':pd.to_datetime(route.get('dateTime')),'latitude':arred(route.get('latitude')),'longitude':arred(route.get('longitude')),'altitude':flt(route.get('altitude'))}, ignore_index=True)
                



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152


In [8]:
# Gravação dos arquivos CSV

atividades.to_csv('Downloads/atividades.csv')
exercicios.to_csv('Downloads/exercicios.csv')
informacoes.to_csv('Downloads/informacoes.csv')
zonas_heart.to_csv('Downloads/zonas_heart.csv')
zonas_speed.to_csv('Downloads/zonas_speed.csv')
amostras_heart.to_csv('Downloads/amostras_heart.csv')
amostras_speed.to_csv('Downloads/amostras_speed.csv')
amostras_distance.to_csv('Downloads/amostras_distance.csv')
amostras_route.to_csv('Downloads/amostras_route.csv')
amostras_altitude.to_csv('Downloads/amostras_altitude.csv')
